In [2]:
import cv2

## Using Haar_Cascade

In [3]:
cascade_path=cv2.data.haarcascades+'haarcascade_frontalface_default.xml'
face_cascade=cv2.CascadeClassifier(cascade_path)

In [3]:
cam=cv2.VideoCapture(0)
face_count=0

while True:
    ret, frame = cam.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        
        if cv2.waitKey(1) & 0xFF ==ord('x'):
            face_image = frame[y:y+h, x:x+w]
            cv2.imwrite(f'face_{face_count}.png', face_image)
            face_count += 1

    # Display the result
    cv2.imshow('Face Detection', frame)
    

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cam.release()
cv2.destroyAllWindows()

## Using DNN

In [ ]:
import numpy as np

# Load the pre-trained face detection model from OpenCV's DNN module
net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')

# Open the default camera (Camera 0)
video = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = video.read()

    # Resize the frame to have a width of 300 pixels (required by the model)
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Set the input to the neural network
    net.setInput(blob)

    # Perform inference and get face detections
    detections = net.forward()

    # Loop over the detections and draw rectangles around the detected faces
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Adjust confidence threshold as needed
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Face Detection', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
video.release()
cv2.destroyAllWindows()
